Astar pseudo code can be found on wiki. 

In my implementation, instead of using an openset, I used an opendict to store the visited vertices and its fscores.

In [1]:
class Vertex:
    
    def __init__(self, node):
        self.id = node
        self.adjacent = {}
        
    def add_neighbor(self, neighbor, weight = 0):
        self.adjacent[neighbor] = weight
        
    def get_neighbors(self):
        return list(self.adjacent.keys())
    
    def get_id(self):
        return self.id
    
    def get_weight(self, neighbor):
        return self.adjacent[neighbor]
    
class NDgraph:
    
    def __init__(self):
        self.vertices = {}
        self.num_vert = 0
        
    def iter(self):
        return iter(self.vertices.values())
    
    def add_vertex(self, nodeid):
        self.vertices[nodeid] = Vertex(nodeid)
        self.num_vert += 1
        
    def get_vertex(self, nodeid):
        if nodeid in self.vertices:
            return self.vertices[nodeid]
        return None
    
    def get_vertices(self):
        return list(self.vertices.keys())
    
    def add_edge(self, fromid, toid, weight):
        if fromid not in self.vertices:
            self.add_vertex(fromid)
        if toid not in self.vertices:
            self.add_vertex(toid)
            
        self.vertices[fromid].add_neighbor(toid, weight)
        self.vertices[toid].add_neighbor(fromid, weight)
    
    def get_edge(self, fromid, toid):
        WRONG = -1
        if not self.get_vertex(fromid) or not self.get_vertex(toid):
            return WRONG
        return self.vertices[fromid].adjacent.get(toid, WRONG)


In [2]:
input_graph = NDgraph()
input_graph.add_vertex('A')
input_graph.add_vertex('B')
input_graph.add_vertex('C')
input_graph.add_vertex('D')
input_graph.add_vertex('E')
input_graph.add_vertex('F')
input_graph.add_vertex('G')
input_graph.add_vertex('H')
input_graph.add_vertex('I')
input_graph.add_vertex('J')

input_graph.add_edge('A', 'B', 6)
input_graph.add_edge('A', 'F', 3)
input_graph.add_edge('B', 'C', 3)
input_graph.add_edge('B', 'D', 2)
input_graph.add_edge('C', 'D', 1)
input_graph.add_edge('C', 'E', 5)
input_graph.add_edge('D', 'E', 8)
input_graph.add_edge('E', 'I', 5)
input_graph.add_edge('E', 'J', 5)
input_graph.add_edge('F', 'G', 1)
input_graph.add_edge('F', 'H', 7)
input_graph.add_edge('G', 'I', 3)
input_graph.add_edge('H', 'I', 2)
input_graph.add_edge('I', 'J', 3)

heuristic = {'A': 10, 'B': 6, 'C': 5, 'D': 7, 'E': 3, 'F': 6, 'G': 5, 'H':3, 'I': 1, 'J': 0}

In [3]:
def Astar(graph: NDgraph, source, target, h: dict):
    """
    A* search algo to find the shortest path from source to target in a graph, return path and distance.
    """
    
    opendict = {source: h[source]} #since python does not support modifying heap values on fly, we use dict instead.
    
    camefrom = {}
    gscore = {nodeid: float('inf') for nodeid in graph.get_vertices()}
    gscore[source] = 0
    
    fscore = {nodeid: float('inf') for nodeid in graph.get_vertices()}
    fscore[source] = h[source]
    
    while len(opendict):
        curr = min(opendict, key = opendict.get)
        
        if curr == target:
            return construct_path(graph, target, source, camefrom)
        
#         print(opendict, curr)
        del opendict[curr]
        
        for neighbor in graph.vertices[curr].get_neighbors():
            temp_gscore = gscore[curr] + graph.get_edge(curr, neighbor)
#             print(neighbor, temp_gscore, gscore[neighbor])
            if temp_gscore < gscore[neighbor]:
                camefrom[neighbor] = curr
                gscore[neighbor] = temp_gscore
                fscore[neighbor] = gscore[neighbor] + h[neighbor]
                opendict[neighbor] = fscore[neighbor]
                    
    return False
        
def construct_path(graph, target, source, camefrom):
    total_dist = 0
    path = []
    curr = target
    while curr != source:
        temp = camefrom[curr]
        total_dist += graph.get_edge(curr, temp)
        path.append(temp)
        curr = temp
    return total_dist, path[::-1] + [target]

In [5]:
Astar(input_graph, 'A', 'J', heuristic)

(10, ['A', 'F', 'G', 'I', 'J'])